In [1]:
import numpy as np
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
import cv2
from keras.utils import image_utils
import datetime

In [2]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    'train',
    target_size=(150,150),
    batch_size=16 ,
    class_mode='binary') #定义训练集

test_set = test_datagen.flow_from_directory(
    'test',
    target_size=(150,150),
    batch_size=16,
    class_mode='binary') #定义测试集

model_saved=model.fit(
    training_set,
    epochs=10,
    validation_data=test_set,
) #定义训练参数

model.save('mymodel.h5',model_saved) #保存训练模型为mymodel.h5

Found 3833 images belonging to 2 classes.
Found 143 images belonging to 2 classes.
Epoch 1/10
142/240 [================>.............] - ETA: 14s - loss: 0.3526 - accuracy: 0.8499

d:\pythonproject\venv\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


240/240 [==============================] - 37s 152ms/step - loss: 0.3077 - accuracy: 0.8750 - val_loss: 0.1156 - val_accuracy: 0.9650
Epoch 2/10
240/240 [==============================] - 34s 141ms/step - loss: 0.1756 - accuracy: 0.9390 - val_loss: 0.0638 - val_accuracy: 0.9930
Epoch 3/10
240/240 [==============================] - 39s 164ms/step - loss: 0.1676 - accuracy: 0.9400 - val_loss: 0.0809 - val_accuracy: 0.9720
Epoch 4/10
240/240 [==============================] - 39s 161ms/step - loss: 0.1728 - accuracy: 0.9410 - val_loss: 0.0897 - val_accuracy: 0.9650
Epoch 5/10
240/240 [==============================] - 39s 161ms/step - loss: 0.1379 - accuracy: 0.9504 - val_loss: 0.0987 - val_accuracy: 0.9720
Epoch 6/10
240/240 [==============================] - 40s 164ms/step - loss: 0.1325 - accuracy: 0.9525 - val_loss: 0.0709 - val_accuracy: 0.9790
Epoch 7/10
240/240 [==============================] - 39s 161ms/step - loss: 0.1049 - accuracy: 0.9611 - val_loss: 0.0256 - val_accuracy: 0.9

In [2]:
mymodel=load_model('mymodel.h5') #加载已保存模型
test_image=image_utils.load_img('test/with_mask/1-with-mask.jpg',
                          target_size=(150,150,3)) #测试模型是否准确
test_image
test_image=image_utils.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]

1/1 [==============================] - 0s 276ms/step


0.0

In [4]:
mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('D:\opencv\haarcascade_frontalface_default.xml') #加载opencv人脸分类器识别人脸

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image_utils.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image_utils.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3) #图片标记
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'HVAE MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)

    cv2.imshow('img',img)

    if cv2.waitKey(1)==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step
